In [ ]:
#|hide
#| eval: false
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
#|default_exp callback.comet

In [ ]:
#|hide
from nbdev.showdoc import *

# Comet.ml

> Integration with [Comet](https://www.comet.com/site/?utm_source=fast_ai&utm_medium=referral&utm_campaign=docs).

## Registration

1. Create account: [comet.com/signup](https://www.comet.com/signup?utm_source=fast_ai&utm_medium=referral&utm_content=docs).
2. Export API key to the environment variable (more help [here](https://www.comet.com/docs/v2/guides/getting-started/quickstart/#get-an-api-key)). In your terminal run:

```
export COMET_API_KEY='YOUR_LONG_API_TOKEN'
```

or include it in your `./comet.config` file (**recommended**). More help is [here](https://www.comet.com/docs/v2/guides/tracking-ml-training/jupyter-notebooks/#set-your-api-key-and-project-name).

## Installation

1. You need to install the Comet SDK. In your terminal run:

```
pip install "comet_ml>=3.37.0"
```

or (alternative installation using conda). In your terminal run:

```
conda install -c anaconda -c conda-forge -c comet_ml comet_ml
```

## How to use?

Start by importing Comet at the top of your training script and then create a new Comet Experiment:

```
import comet_ml

experiment = comet_ml.Experiment(project_name="my-fastai-project")
```

Comet logs your experiment through a callback executed when you run Learner.fit() in fastai. You do not need to add this callback yourself, we do it for you automatically. However, if you ever need to access the callback manually, you can do it the following way:

```
import comet_ml

experiment = comet_ml.Experiment()

comet_ml_callback = experiment.get_callback("fastai")

learn = Learner(dls, model,
                cbs=comet_ml_callback
                )
```